Чтение данных из различных источников и формирования pkl файлов в папке interm/external_pkl, а также 
Извлечение pkl файлов исходных (data/external) для дальнейшего использования    

Делать однократно, если ранее не были созданы pkl файлы в папке interm/external_pkl  

В проекте в основном необходимые входные файлы data/external лежат в заархивированном виде     
сами pkl созданы на версии pandas 1.3.5  

In [1]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки
%run ../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init_mdm.ipynb
%run ../preprocessing/preprocessing_target.ipynb

In [2]:
def read_codes_ext(name):
    df_codes = (pd.read_excel(name, engine='openpyxl', dtype = {"Код":str} )
                        .dropna().rename(columns={'Наименование':'ClassifierName', 'Код': 'content_FinancialLeaseSubject_ClassifierCode'}))
    print("Прочитанный размер файла классификатора имущества - {}".format(df_codes.shape))
    
    return df_codes

df_codes = read_codes_ext(pre+"data/external/Классификатор имущества.xlsx")
df_codes.to_pickle(pre+"data/interm/external_pkl/Классификатор имущества.pkl", compression={'method':'zip'})


Прочитанный размер файла классификатора имущества - (168, 2)


In [3]:
def read_target_ext():
    
    name = pre + 'data/external/target_data.tar.gz'
    extract_all(name, pre +'data/interm/external_pkl')

    name = pre + "data/interm/external_pkl/dataset_local_application_inf_ssf.01.07.2022.pkl"
    target_data = pd.read_pickle(name)

    print("Прочитанный размер файла из target-class {}".format(target_data.shape))
    target_data = target_filter(target_data)
    
    try:
        if os.path.isfile(name):
            os.remove(name)
    except:
        pass

    return target_data

target_data = read_target_ext()
target_data.to_pickle(pre+"data/interm/external_pkl/target_dataset.pkl", compression={'method':'zip'})
# target_data.head(1)

Прочитанный размер файла из target-class (252134, 94)


In [4]:
def read_financialleasecontract_ext():
    sql_query = """select * from [dbo].[efrsfdul_financialleasecontract]"""

    cnxn = pymssql.connect(host='VLS-MDM-DB',user=mssql_user, password=mssql_password,  database='MDM_Main', charset='utf8')

    try:
        financialleasecontract = pd.read_sql(sql_query, cnxn)
        print("Размер считанных данных financialleasecontract - {}".format(financialleasecontract.shape))

    finally:
        cnxn.close() 
        
    return financialleasecontract

financialleasecontract = read_financialleasecontract_ext()
financialleasecontract.to_pickle(pre+"data/interm/external_pkl/financialleasecontract.pkl", compression={'method':'zip'})



Размер считанных данных financialleasecontract - (1493510, 20)


In [5]:
def read_financialleasecontractlessees_ext():
    sql_query = """select * from [dbo].[efrsfdul_financialleasecontractlessees]"""

    cnxn = pymssql.connect(host='VLS-MDM-DB',user=mssql_user, password=mssql_password,  database='MDM_Main', charset='utf8')

    try:
        financialleasecontractlessees = pd.read_sql(sql_query, cnxn)
        print("Размер считанных данных financialleasecontractlessees - {}".format(financialleasecontractlessees.shape))

    finally:
        cnxn.close() 
        
    return financialleasecontractlessees

financialleasecontractlessees = read_financialleasecontractlessees_ext()
financialleasecontractlessees.to_pickle(pre+"data/interm/external_pkl/financialleasecontractlessees.pkl", compression={'method':'zip'})

Размер считанных данных financialleasecontractlessees - (1489613, 12)


In [6]:
def read_financialleasecontractlessors_ext():
    sql_query = """select * from [dbo].[efrsfdul_financialleasecontractlessors]"""

    cnxn = pymssql.connect(host='VLS-MDM-DB',user=mssql_user, password=mssql_password,  database='MDM_Main', charset='utf8')

    try:
        financialleasecontractlessors = pd.read_sql(sql_query, cnxn)
        print("Размер считанных данных financialleasecontractlessors - {}".format(financialleasecontractlessors.shape))

    finally:
        cnxn.close() 
        
    return financialleasecontractlessors

financialleasecontractlessors = read_financialleasecontractlessors_ext()
financialleasecontractlessors.to_pickle(pre+"data/interm/external_pkl/financialleasecontractlessors.pkl", compression={'method':'zip'})

Размер считанных данных financialleasecontractlessors - (1418815, 12)


In [7]:
def read_financialleasecontractobject_ext():
    sql_query = """select * from [dbo].[efrsfdul_financialleasecontractobject]"""

    cnxn = pymssql.connect(host='VLS-MDM-DB',user=mssql_user, password=mssql_password,  database='MDM_Main', charset='utf8')

    try:
        financialleasecontractobject = pd.read_sql(sql_query, cnxn)
        print("Размер считанных данных financialleasecontractobject - {}".format(financialleasecontractobject.shape))

    finally:
        cnxn.close() 
        
    return financialleasecontractobject

financialleasecontractobject = read_financialleasecontractobject_ext()
financialleasecontractobject.to_pickle(pre+"data/interm/external_pkl/financialleasecontractobject.pkl", compression={'method':'zip'})

Размер считанных данных financialleasecontractobject - (3298676, 12)


In [2]:
def read_accounts_from_crm():
    
    cnxn = pymssql.connect(host='ALS-CRM-SQL\DEV01',user=mssql_user, password=mssql_password,  database='VTBL_MSCRM', charset='utf8')

    #  запускать 1 раз
    sql_query =   """ 
                    SELECT a.ACCOUNTID,
                        a.Name,
                        a.NEW_INN,
                        a.NEW_NUM1C,
                        a.new_black_list_date, 
						a.new_4reliability_group,
                        a.StatusCode,
                        s1.Value AS StatusCodeName,
                        a.new_suptype,
                        s2.Value AS SupTypeName 
                        from VTBL_MSCRM.dbo.AccountBase as a
                        left join VTBL_MSCRM.dbo.StringMap as s1 on s1.AttributeValue=a.StatusCode and s1.AttributeName='statuscode' and s1.ObjectTypeCode=1
                        left join VTBL_MSCRM.dbo.StringMap as s2 on s2.AttributeValue=a.new_suptype and s2.AttributeName='new_suptype' and s2.ObjectTypeCode=1 
                        --WHERE ACCOUNTID='3F957807-AC32-ED11-A2CF-005056AB1749'
                    """
    try:
        data_accounts_crm = pd.read_sql(sql_query, cnxn)
    finally:
        cnxn.close() 
    
      
    print("Всего выгружено контрагентов из crm ", data_accounts_crm.shape)

    return data_accounts_crm

data_accounts_crm = read_accounts_from_crm()
data_accounts_crm.to_pickle(pre+"data/interm/external_pkl/crm_account.pkl", compression={'method':'zip'})
# data_accounts_crm.head(1)

Всего выгружено контрагентов из crm  (765391, 10)


In [3]:
# [egrul_regdata] и  [egrip_regdata]
def read_regdata_from_mdm():
    
    cnxn = pymssql.connect(host='VLS-MDM-DB-N1', user=mssql_user, password=mssql_password,  database='MDM_Main', charset='utf8')

    #  запускать 1 раз
    sql_query =   """ 
                    SELECT [new_inn]
                        ,[new_typeofdissolution]
                        ,[new_dissolutiondate]
                        , [new_accounstatus]
                    from [MDM_Main].[dbo].[egrul_regdata]
                    where [new_typeofdissolution] IS NOT NULL OR
                    [new_dissolutiondate] IS NOT NULL OR
                    [new_accounstatus] IS NOT NULL
                    UNION ALL 
                    (SELECT [new_inn]
                        ,[new_typeofdissolution]
                        ,[new_dissolutiondate]
                        , NULL
                    from [MDM_Main].[dbo].[egrip_regdata]
                    where [new_typeofdissolution] IS NOT NULL OR
                    [new_dissolutiondate] IS NOT NULL
                    )
                    """
    try:
        regdata_from_mdm = pd.read_sql(sql_query, cnxn)
    finally:
        cnxn.close() 
    
      
    print("Всего выгружено контрагентов из regdata mdm ", regdata_from_mdm.shape)

    return regdata_from_mdm

regdata_from_mdm = read_regdata_from_mdm()
regdata_from_mdm.to_pickle(pre+"data/interm/external_pkl/regdata_from_mdm.pkl", compression={'method':'zip'})
# regdata_from_mdm.head(1)

Всего выгружено контрагентов из regdata mdm  (43884, 4)


In [4]:
def read_clients_in_behavior_model_daily():
    
    cnxn_oracle = cx_Oracle.connect(obi_login, obi_pw, "10.0.2.45:1521/UAKRPDB", encoding="UTF-8")  
    #  запускать 1 раз
    sql_query =   """ 
                    SELECT DISTINCT cl.INN inn_in_bmd
                    FROM DM.BEHAVIOR_MODEL_DAILY bmd
                    INNER JOIN
                        (SELECT INN,
                            KPP,
                            OGRN,
                            CLIENT_KEY
                        FROM DWH.Clients
                        WHERE VALID_TO_DTTM    = TO_DATE('01.01.2400', 'dd.mm.yyyy')
                        AND NVL(DELETE_FLG, 0)!=1
                        ) cl
                    ON cl.CLIENT_KEY = bmd.CLIENT_KEY
                    WHERE cl.INN IS NOT NULL 
                    """
    try:
        data_clients_bmd = pd.read_sql(sql_query, cnxn_oracle)
    finally:
        cnxn_oracle.close() 
    
    print("Всего выгружено контрагентов из DM.BEHAVIOR_MODEL_DAILY ", data_clients_bmd.shape)

    return data_clients_bmd

data_clients_bmd = read_clients_in_behavior_model_daily()
data_clients_bmd.to_pickle(pre+"data/interm/external_pkl/clients_bmd.pkl", compression={'method':'zip'})

Всего выгружено контрагентов из DM.BEHAVIOR_MODEL_DAILY  (95002, 1)


In [5]:
def read_inn_opportunity_ext():
    sql_query = """SELECT op.name,  a.NEW_INN, op.new_opt
                        FROM dbo.Opportunity op
                    LEFT JOIN
                        (SELECT *
                            FROM ACCOUNT
                        ) AS a
                    ON op.customerid=a.accountid 
                    --WHERE  op.name= 'АЛ933631'
                    """

    cnxn = pymssql.connect(host='ALS-CRM-SQL\DEV01',user=mssql_user, password=mssql_password,  database='VTBL_MSCRM', charset='utf8')

    try:
        opportunity_CRM = pd.read_sql(sql_query, cnxn)
        opportunity_CRM.columns = ['opp_name',	'ИНН контрагента',	'new_opt']
        print("Размер считанных данных dbo.Opportunity- {}".format(opportunity_CRM.shape))

    finally:
        cnxn.close() 
        
    return opportunity_CRM

opportunity_CRM = read_inn_opportunity_ext()
opportunity_CRM.to_pickle(pre+"data/interm/external_pkl/opportunity_crm.pkl", compression={'method':'zip'})

Размер считанных данных dbo.Opportunity- (1077778, 3)


In [6]:
def MDL_APP_DAKSENOVA():
    
    cnxn_oracle = cx_Oracle.connect(obi_login, obi_pw, "10.0.2.45:1521/UAKRPDB", encoding="UTF-8") 
    
    sql_query = """select pl,  
                        DATE_ACTUAL, 
                        NL_NEW_PRODUCTNAME, 
                        O_NEW_TOTALFINANCINGSUM 
                    from ANALYST.MDL_APP_DAKSENOVA_010723
                    """

    try:
        mdl_app = pd.read_sql(sql_query, cnxn_oracle)
        print("Размер считанных данных mpl_app - {}".format(mdl_app.shape))

    finally:
        cnxn_oracle.close() 
        
    return mdl_app

mdl_app = MDL_APP_DAKSENOVA()
mdl_app.to_pickle(pre+"data/interm/external_pkl/mdl_app.pkl", compression={'method':'zip'})
# mdl_app.head()

Размер считанных данных mpl_app - (250698, 4)
